# Assignment 2
Welcome to the second programming assigment for the course. This assignments will help to familiarise you with entangled states while revisiting the topics discussed in this week's lectures. 
<div class="alert alert-block alert-warning"><b>Convention: </b>All operations and circuits should be constructed in the big endian representation. </div>

### Submission Guidelines
For final submission, and to ensure that you have no errors in your solution, please use the 'Restart and Run All' option availble in the Kernel menu at the top of the page. 
To submit your solution, run the completed notebook and attach the solved notebook (with results visible) as a file using the 'Add or Create' option under the 'Your Work' heading on the assignment page in Google Classroom. 

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Importing standard Qiskit libraries and configuring account
import qiskit
from qiskit import QuantumCircuit, execute
from qiskit.providers.aer import QasmSimulator, StatevectorSimulator
from qiskit.visualization import *
from qiskit.quantum_info import *
success_msg = 'Your answer is correct.'
fail_msg = 'Your answer is not correct. Please try again.'
basis_gates = ['id', 'x', 'y', 'z', 's', 't', 'sdg', 'tdg', 'h', 'p', 'sx' ,'r', 'rx', 'ry', 'rz', 'u', 'u1', 'u2', 'u3', 'cx', 'barrier', 'measure', 'snapshot']

## Entanglement
Multi-partite entangled states are also possible. A very popular example of such a state is the  GHZ state, named after the authors who first proposed it's interesting properties. This belongs to a class of multi-qubit states called _cat states_, after Schrodinger's cat. You will create this state in the first problem.
## **Problem 1**
Prepare the GHZ state $|\text{GHZ}\rangle = \frac{1}{\sqrt{2}}(|000\rangle + |111\rangle)$ in a quantum circuit **using only the basic single-qubit gates and CNOT**. Below we have provided you with some code to create a quantum circuit. Remember that a qubit in a quantum circuit always begins in the $|0\rangle$ state. Add appropriate gates. A standard basis measurement has already been added for you.

In [ ]:
qc1 = QuantumCircuit(3) 
# Insert gates below to create the state 

# Do not change below this line. You do not need to add an additional measurement. 
qc1.measure_all()
qc1.draw('mpl')

<div class="alert alert-block alert-info"><b>Instructions:</b> Once your circuit is ready, run the cell below to check your answer. </div>

In [ ]:
try:
    assert list(qc1.count_ops()) != [], "Circuit cannot be empty"
    assert set(qc1.count_ops().keys()).difference(basis_gates) == set(), f"Only the following basic gates are allowed: {basis_gates}"
    assert all([type(gate[0]) == qiskit.circuit.measure.Measure for gate in qc1.data[-3:len(qc1.data)]]), "Measurement must be the last operation in a circuit."
    sv_check = Statevector.from_instruction(qc1.remove_final_measurements(inplace=False)).equiv((Statevector.from_label('000')+Statevector.from_label('111'))/np.sqrt(2))
    assert sv_check, "You did not prepare the correct state."
    job = execute(qc1, backend=QasmSimulator(), shots=1024, seed_simulator=0)
    counts = job.result().get_counts()
    print(success_msg if (sv_check) else fail_msg)  
except AssertionError as e:
    print(f'Your code has an error:  {e.args[0]}')
    
except Exception as e:
    print(f'This error occured: {e.args[0]}')
plot_histogram(counts)

## Superdense Coding
In quantum information, there are various measures and methods to compare the amount of information that can be stored or transmitted using qubits and how that compares to classical bits. While these are far outside the scope of an introductory course, a proof of concept depicting an enhancement facilitated by quantum entanglement should be simple enough to see. Superdense coding (sometimes also called dense coding) is one such example. It is also very closely related to quantum teleportation. The protocol is detailed below.

Using superdense coding, two parties (Alice and Bob) who share an entangled Bell pair, can transmit two classical bits worth of information by sending only one qubit from the sender to the recipient. The important part here is really the Bell pair as this protocol does not work without it. In the case presented below, Alice is the sender and Bob is the recipient. 

### Protocol:

1. Alice prepares a Bell state $|B_{00}\rangle$ and transmits the second qubit to Bob. (Note that no 'information' has been encoded or sent yet).
2. **The Encoding Step:** Alice encodes two classical bits of information into her qubit (one of the Bell pair qubits) by performing only single-qubit operations.
3. Alice then sends her qubit to Bob.
4. **The Decoding Step:** Bob measures both the qubits and gets both of Alice's encoded bits as the measurement outcome.

In this problem, we will _simulate_ superdense coding in the framework of quantum computing. Because of this, it is not possible to send any qubits anywhere, so we will have to pretend those parts are happening. We will imagine that Alice prepares the Bell pair and sends the second qubit to Bob. As we mentioned before, no information has been sent yet because if Bob measures his qubit now, he will measure equal outcomes in any basis he chooses to measure in. Alice then perform her encoding operations only on the first qubit. Our understanding will be that she cannot perform any operations on the second qubit since she has already sent it to Bob. Now we imagine that she sends her qubit to Bob so that he now has both qubits. Bob can now measure both qubits in the Bell basis and it is expected that he will be able to decode the bits Alice encoded. We will simulate this entire process using a two-qubit quantum circuit. 

## **Problem 2**

Given below is the structure for a function `superdense_coding_circ(bitstr)`. This will generate the quantum circuit for the whole process when Alice wants to encode and transmit the two-bit string`bitstr` (`bitstr` has possible values `00`,`01`,`10`,`11`). Your task is to complete the function to behave as expected. You may use the `compose()` ([documentation](https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.compose.html#qiskit.circuit.QuantumCircuit.compose)) and `extend()`([documentation](https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.extend.html#qiskit.circuit.QuantumCircuit.extend)) to join circuits together. Please DO NOT use `append()` as it will render the appended operations as opaque gates and your answer will be treated as incorrect. 

<div class="alert alert-block alert-warning"><b>Instructions:</b> To check your answers, we take a few snapshots of your circuit. Please do not remove these commands, as this will cause your solution to be treated as incorrect. Each of the following steps must be followed before the corresponding snapshot.</div> 

1. Prepare the Bell state $|B_{00}\rangle$.
2. Perform the single-qubit encoding operation on the first qubit using basic single qubit gates only. The single qubit operations should be such that it changes the $|B_{00}\rangle$ to the Bell state corresponding to `bitstr`. For example, if Alice wants to encode the `bitstr` `01`, any single qubit operations you apply on the first qubit should take the two-qubit state from $|B_{00}\rangle$ to $|B_{01}\rangle$. This should work for all four possible values of `bitstr`.
3. Apply the necessary gates to switch to the Bell basis for measurement. A measurement instruction is provided for you, so please do not add another one. 

The four Bell states are given below:

\begin{equation*}
\begin{split}
|B_{00}\rangle &\,=\, \frac{|00\rangle + |11\rangle}{\sqrt{2}} \\
|B_{01}\rangle &\,=\, \frac{|01\rangle + |10\rangle}{\sqrt{2}} \\
|B_{10}\rangle &\,=\, \frac{|00\rangle - |11\rangle}{\sqrt{2}} \\
|B_{11}\rangle &\,=\, \frac{|01\rangle - |10\rangle}{\sqrt{2}}   
\end{split}
\end{equation*}

In [ ]:
def superdense_coding_circ(bitstr):
    '''
    bitstr is a 2-character string with possible values '00', '01', '10', '11'
    '''
    qc2 = QuantumCircuit(2)
    # Enter your code below this line
    # Prepare a Bell state B_00 below
    
    # Do not remove the following line
    qc2.snapshot('1')
    # Apply single-qubit gates only on q_0 to encode the bitstring

    # Do not remove the following line
    qc2.snapshot('2')
    # Apply gates to change to the Bell basis

    # Do not remove the following line   
    qc2.snapshot('3')
    # Do not change below this line. You do not need to add an additional measurement. 
    qc2.measure_all()
    return qc2

<div class="alert alert-block alert-info"><b>Instructions:</b> Run the cell below to check your answer. </div>

In [2]:
b00_sv = (Statevector.from_label('00')+Statevector.from_label('11'))/np.sqrt(2)
b01_sv = (Statevector.from_label('01')+Statevector.from_label('10'))/np.sqrt(2)
b10_sv = (Statevector.from_label('00')-Statevector.from_label('11'))/np.sqrt(2)
b11_sv = (Statevector.from_label('01')-Statevector.from_label('10'))/np.sqrt(2)
bell_states_sv = [b00_sv, b01_sv, b10_sv, b11_sv]
basis_states = ['00', '01', '10', '11']
bell_dict = dict(zip(basis_states, bell_states_sv))
enc_dec_mat = np.zeros((len(basis_states),len(basis_states)))
try:
    for state in basis_states:
        sv_checks = []
        qc = superdense_coding_circ(state)
        assert list(qc.count_ops()) != [], f" Bitstring {state}: Circuit cannot be empty"
        assert qc.count_ops()['measure'] <= 2, f" Bitstring {state}: Please remove extra measurements"
        assert qc.count_ops()['snapshot'] == 3, f" Bitstring {state}: Please do not add or remove snapshots"
        assert set(qc.count_ops().keys()).difference(basis_gates) == set(), f" Bitstring {state}: Only the following basic gates are allowed: {basis_gates}"
        assert all([type(gate[0]) == qiskit.circuit.measure.Measure for gate in qc.data[-2:len(qc.data)]]), f" Bitstring {state}: Measurement must be the last operation in a circuit."
        result = execute(qc.reverse_bits(),backend=QasmSimulator(), shots=1).result().get_counts().most_frequent()
        enc_dec_mat[int(state,2), int(result,2)] = 1
        sv = execute(qc.reverse_bits(), backend=StatevectorSimulator()).result().data()['snapshots']['statevector']
        sv_checks.append(b00_sv.equiv(sv['1'][0]))
        sv_checks.append(bell_dict[state].equiv(sv['2'][0]))
        sv_checks.append(Statevector.from_label(state).equiv(sv['3'][0]))
    assert all(np.diag(enc_dec_mat) == np.ones(4)), "Your encoding is not correct"
    plt.matshow(enc_dec_mat, cmap='binary')
    plt.title('Superdense Coding Matrix')
    plt.xlabel("Alice's encoded bits", fontsize=14)
    plt.ylabel("Bob's decoded bits", fontsize=14)
    plt.xticks(ticks=range(4), labels=basis_states, fontsize=12)
    plt.yticks(ticks=range(4), labels=basis_states, fontsize=12)
    circ_check = all(sv_checks)
    assert circ_check, "Your circuit does not work for all bitstrings"

    print(success_msg if circ_check else fail_msg)
except AssertionError as e:
    print(f'Your code has an error:  {e.args[0]}')
    
except Exception as e:
    print(f'This error occured: {e.args[0]}')


NameError: name 'Statevector' is not defined